In [1]:
#imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from urllib.request import urlopen
from bs4 import BeautifulSoup
from sys import platform
import io
import datetime

if platform == "linux" or platform == "linux2":
    data_string = "/home/maria/Documents/data/pied_piper/puyallup/"
    output_string = "/home/maria/Documents/output/pied_piper/puyallup/"

elif platform == "win32"or platform == "win64":
    data_string = "C:/Users/maria/OneDrive/Documents/data/pied_piper/puyallup/"
    output_string = "C:/Users/maria/OneDrive/Documents/output/pied_piper/puyallup/"

In [2]:
# #read file in Windows
try:
    df = pd.read_excel(
   io=data_string + "puyallup_chinook_2004-2018.xlsx",
   sheet_name ='2010_edited',
    header = [3]
)
except PermissionError:
    print("Permission Error. Close the file, if it is open.")
    

df.head()

,Date,flow,In_day,secchi_depth_day,chinook0_wild_num_day,trap_efficiency,chinook0_wild_num_day_estimate,In_night,secchi_depth_night,chinook0_wild_num_night,...,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,chinook0_hatchery_num_night,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,chinook0_hatchery_num
0,2010-01-21,1400.0,-,-,-,0.0259,0.000000,15.75,222,1,...,NaN,NaN,15.75,222,0,NaN,NaN,NaN,NaN,0.0
1,2010-01-22,1270.0,9.25,220,1,0.0259,38.610039,15,210,0,...,NaN,NaN,15,210,0,NaN,NaN,NaN,NaN,0.0
2,2010-01-23,1180.0,8.75,212,0,0.0259,0.000000,13.75,216,2,...,NaN,NaN,13.75,216,0,NaN,NaN,NaN,NaN,0.0
3,2010-01-24,1170.0,9,155,0,0.0259,0.000000,14.5,123,0,...,NaN,NaN,14.5,123,0,NaN,NaN,NaN,NaN,0.0
4,2010-01-25,1600.0,9.75,155,5,0.0259,193.050193,13.5,186,0,...,NaN,NaN,13.5,186,0,NaN,NaN,NaN,NaN,0.0


In [5]:
pd.unique(df['In_day']) # '-',nan
pd.unique(df['In_night'])# '-',nan

df.columns

Index(['Date', 'flow', 'In_day', 'secchi_depth_day', 'chinook0_wild_num_day',
       'trap_efficiency', 'chinook0_wild_num_day_estimate', 'In_night',
       'secchi_depth_night', 'chinook0_wild_num_night',
       'trap_efficiency_night', 'chinook0_wild_num_night_estimate', 'In',
       'secchi_depth', 'chinook0_wild_num', 'chinook0_wild_num_estimate',
       'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19',
       'Unnamed: 20', 'Unnamed: 21', 'chinook0_hatchery_num_day',
       'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26',
       'Unnamed: 27', 'chinook0_hatchery_num_night', 'Unnamed: 29',
       'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'chinook0_hatchery_num'],
      dtype='object')

In [6]:
df_subset = df[['Date', 'flow', 'secchi_depth_day','In_day','In_night','In', 'secchi_depth_night',
                'trap_efficiency',
                'chinook0_wild_num_day',
                'chinook0_wild_num_night','chinook0_wild_num',
                'chinook0_hatchery_num_day',
                'trap_efficiency_night',
                'chinook0_hatchery_num_night','chinook0_hatchery_num',
                'chinook0_wild_num_day_estimate','chinook0_wild_num_night_estimate']]

In [7]:
df_subset = df_subset.replace({'-':np.nan})
df_subset.loc[pd.isna(df_subset['In_day']),'chinook0_wild_num_day'] =np.nan
df_subset.loc[pd.isna(df_subset['In_day']),'chinook0_wild_num_day_estimate'] =np.nan
df_subset.loc[pd.isna(df_subset['In_day']),'chinook0_hatchery_num_day'] =np.nan

df_subset.loc[pd.isna(df_subset['In_night']),'chinook0_wild_num_night'] =np.nan
df_subset.loc[pd.isna(df_subset['In_night']),'chinook0_wild_num_night_estimate'] =np.nan
df_subset.loc[pd.isna(df_subset['In_night']),'chinook0_hatchery_num_night'] =np.nan

In [14]:
#if trap was out for both day and night, the total values should be zero
df_subset.loc[pd.isna(df_subset['In_night']),'chinook0_hatchery_num'][pd.isna(df_subset['In_day'])] = np.nan
df_subset.loc[pd.isna(df_subset['In_night']),'chinook0_wild_num'][pd.isna(df_subset['In_day'])] = np.nan

In [15]:
df_subset['chinook0_wild_perhour_day'] = df_subset['chinook0_wild_num_day']/df_subset['In_day']
df_subset['chinook0_wild_perhour_night'] = df_subset['chinook0_wild_num_night']/df_subset['In_night']
df_subset['chinook0_wild_perhour'] = df_subset['chinook0_wild_num']/df_subset['In']

df_subset['chinook0_hatchery_perhour_day'] = df_subset['chinook0_hatchery_num_day']/df_subset['In_day']
df_subset['chinook0_hatchery_perhour_night'] = df_subset['chinook0_hatchery_num_night']/df_subset['In_night']
df_subset['chinook0_hatchery_perhour'] = df_subset['chinook0_hatchery_num']/df_subset['In']

# df_subset['secchi_depth_day'] = df_subset['secchi_depth']

df_subset['flow_day'] = df_subset['flow']
df_subset['flow_night'] = df_subset['flow']

In [17]:
df_subset['chinook0_wild_perhour_day_estimate'] = (df_subset['chinook0_wild_num_day_estimate']/
                                                   df_subset['In_day'])
df_subset['chinook0_wild_perhour_night_estimate'] = (df_subset['chinook0_wild_num_night_estimate']/
                                                   df_subset['In_night'])
df_subset['chinook0_wild_perhour_estimate'] = ((df_subset['chinook0_wild_num_day_estimate']+df_subset['chinook0_wild_num_night_estimate'])/
                                                   df_subset['In'])


df_subset['chinook0_hatchery_num_day_estimate'] = df_subset['chinook0_hatchery_num_day']/df_subset['trap_efficiency']
df_subset['chinook0_hatchery_num_night_estimate'] = df_subset['chinook0_hatchery_num_night']/df_subset['trap_efficiency']


df_subset['chinook0_hatchery_perhour_day_estimate'] = (df_subset['chinook0_hatchery_num_day_estimate']/
                                                   df_subset['In_day'])
df_subset['chinook0_hatchery_perhour_night_estimate'] = (df_subset['chinook0_hatchery_num_night_estimate']/
                                                   df_subset['In_night'])
df_subset['chinook0_hatchery_perhour_estimate'] = ((df_subset['chinook0_hatchery_num_day_estimate']+df_subset['chinook0_hatchery_num_night_estimate'])/
                                                   df_subset['In'])

In [21]:
df_subset['chinook0_wild_day_fraction'] = np.nan
df_subset['chinook0_hatchery_day_fraction'] = np.nan
n = len(df_subset['chinook0_wild_perhour'])

for i in range(n):
    if df_subset['chinook0_wild_perhour_day'].loc[i]==0 and df_subset['chinook0_wild_perhour_night'].loc[i] == 0:
        df_subset['chinook0_wild_day_fraction'].loc[i] = np.nan
        
    else:
        df_subset['chinook0_wild_day_fraction'].loc[i] = (df_subset['chinook0_wild_perhour_day'].loc[i]/
                                                          (df_subset['chinook0_wild_perhour_day'].loc[i] + df_subset['chinook0_wild_perhour_night'].loc[i]))
        
    if df_subset['chinook0_hatchery_perhour_day'].loc[i]==0 and df_subset['chinook0_hatchery_perhour_night'].loc[i] == 0:
        df_subset['chinook0_hatchery_day_fraction'].loc[i] = np.nan
        
    else:
        df_subset['chinook0_hatchery_day_fraction'].loc[i] = (df_subset['chinook0_hatchery_perhour_day'].loc[i]/
                                                          (df_subset['chinook0_hatchery_perhour_day'].loc[i] + df_subset['chinook0_hatchery_perhour_night'].loc[i]))
        
        

/home/maria/miniconda3/envs/analysis/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [22]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_subset)

,Date,flow,secchi_depth_day,In_day,In_night,In,secchi_depth_night,trap_efficiency,chinook0_wild_num_day,chinook0_wild_num_night,chinook0_wild_num,chinook0_hatchery_num_day,trap_efficiency_night,chinook0_hatchery_num_night,chinook0_hatchery_num,chinook0_wild_num_day_estimate,chinook0_wild_num_night_estimate,chinook0_wild_perhour_day,chinook0_wild_perhour_night,chinook0_wild_perhour,chinook0_hatchery_perhour_day,chinook0_hatchery_perhour_night,chinook0_hatchery_perhour,flow_day,flow_night,chinook0_wild_perhour_day_estimate,chinook0_wild_perhour_night_estimate,chinook0_wild_perhour_estimate,chinook0_hatchery_num_day_estimate,chinook0_hatchery_num_night_estimate,chinook0_hatchery_perhour_day_estimate,chinook0_hatchery_perhour_night_estimate,chinook0_hatchery_perhour_estimate,chinook0_wild_day_fraction,chinook0_hatchery_day_fraction
0,2010-01-21,1400.0,NaN,NaN,15.75,15.75,222.0,0.0259,NaN,1.0,1.0,NaN,0.035469,0.0,0.0,NaN,28.193837,NaN,0.063492,0.063492,NaN,0.000000,0.000000,1400.0,1400.0,NaN,1.790085,NaN,NaN,0.000000,NaN,0.000000,NaN,NaN,NaN
1,2010-01-22,1270.0,220.0,9.25,15.00,24.25,210.0,0.0259,1.0,0.0,1.0,0.0,0.032409,0.0,0.0,38.610039,0.000000,0.108108,0.000000,0.041237,0.000000,0.000000,0.000000,1270.0,1270.0,4.174058,0.000000,1.592167,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,NaN
2,2010-01-23,1180.0,212.0,8.75,13.75,22.50,216.0,0.0259,0.0,2.0,2.0,0.0,0.030101,0.0,0.0,0.000000,66.443706,0.000000,0.145455,0.088889,0.000000,0.000000,0.000000,1180.0,1180.0,0.000000,4.832270,2.953054,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
3,2010-01-24,1170.0,155.0,9.00,14.50,23.50,123.0,0.0259,0.0,0.0,0.0,0.0,0.029833,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1170.0,1170.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN
4,2010-01-25,1600.0,155.0,9.75,13.50,23.25,186.0,0.0259,5.0,0.0,5.0,0.0,0.039662,0.0,0.0,193.050193,0.000000,0.512821,0.000000,0.215054,0.000000,0.000000,0.000000,1600.0,1600.0,19.800020,0.000000,8.303234,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,NaN
5,2010-01-26,1340.0,NaN,10.00,13.75,23.75,209.0,0.0259,0.0,1.0,1.0,0.0,0.034093,0.0,0.0,0.000000,29.331240,0.000000,0.072727,0.042105,0.000000,0.000000,0.000000,1340.0,1340.0,0.000000,2.133181,1.235000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
6,2010-01-27,1210.0,225.0,9.75,13.75,23.50,217.0,0.0259,0.0,0.0,0.0,0.0,0.030889,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1210.0,1210.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN
7,2010-01-28,1110.0,217.0,7.50,16.25,23.75,221.0,0.0259,0.0,0.0,0.0,0.0,0.028180,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1110.0,1110.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN
8,2010-01-29,1040.0,210.0,6.50,16.50,23.00,210.0,0.0259,0.0,0.0,0.0,0.0,0.026135,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1040.0,1040.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN
9,2010-01-30,1070.0,215.0,9.50,14.25,23.75,171.0,0.0259,0.0,3.0,3.0,0.0,0.027028,0.0,0.0,0.000000,110.996015,0.000000,0.210526,0.126316,0.000000,0.000000,0.000000,1070.0,1070.0,0.000000,7.789194,4.673516,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN


In [23]:
#save the data

df_subset.to_csv(data_string + "2010_puyallup.csv")  